<a href="https://colab.research.google.com/github/LuisAdorno/TrabalhoRNLuis/blob/main/Luis_TrabalhoRN1FinalVGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vamos Classificar RX de covid e normais

In [ ]:
!pip install tensorflow==2.8.0

In [ ]:
!pip install protobuf==3.20

In [ ]:
import sys
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import glob
from imutils import paths

Vamos a procurar o caminho de nossa base

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive'
os.chdir(workdir_path)

#Vamos colocar os parametros de nosso modelo

In [ ]:
INIT_LR = 1e-3
EPOCHS = 15 # PODE MUDAR
BS = 8

# Pega a lista de imagens no nosso diretório de conjunto de dados e, em seguida, inicializa
# a lista de dados e as respectivas classes das imagens

In [ ]:

dataset_path = 'dataset'
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset_path))
data = []
labels = []

In [ ]:
# recorrer las rutas de la imagen
for imagePath in imagePaths:
    # extrae la etiqueta de clase del nombre del archivo
    label = imagePath.split(os.path.sep)[-2]

   # cargue la imagen, intercambie canales de color y cambie su tamaño para que sea una
   # 224x224 píxeles sin tener en cuenta la relación de aspecto
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # actualizar las listas de datos y etiquetas, respectivamente
    data.append(image)
    labels.append(label)
print("labels: ", np.unique(labels))

In [ ]:
# convierta los datos y las etiquetas en matrices NumPy mientras escala el píxel
# intensidades en el rango [0, 255]
data = np.array(data) / 255.0
labels = np.array(labels)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(data[labels=='covid'][1])
plt.show()

In [ ]:
# realizar una codificación one-hot en las etiquetas
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
# particione los datos en divisiones de entrenamiento y prueba usando el 80% de
# los datos para entrenamiento y el 20% restante para pruebas
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.20, stratify=labels, random_state=42)

## Data Augmentation

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [ ]:
# cargue la red VGG16, asegurándose de que queden los conjuntos de capas FC principales
# off
baseModel = VGG16(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
#headModel = MaxPooling2D(pool_size=(2, 2))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)  # Replace 'decay' with 'learning_rate'
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

In [ ]:
baseModel = VGG16(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))

f1 = baseModel.layers[1].output
f2 = baseModel.layers[2].output
f3 = baseModel.layers[4].output
f4 = baseModel.layers[5].output
feature_maps = Model(inputs=baseModel.input, outputs=[f1, f2, f3, f4])

In [ ]:
y = np.argmax(trainY, axis=-1)

In [ ]:
feat1, feat2, feat3, feat4 = feature_maps.predict(trainX[y==1][0:1])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].imshow(feat1[0, :, :, 0:3])
axs[0, 1].imshow(feat2[0, :, :, 6:9])
axs[1, 0].imshow(feat3[0, :, :, 0:3])
axs[1, 1].imshow(feat4[0, :, :, 3:6])
# plt.subplots
# plt.imshow(feat1[0, :, :, 0:3])
plt.show()

##passo os dados de entrada

In [ ]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
    trainAug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
                            target_names=lb.classes_))

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testY.argmax(axis=1), predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()
# plt.savefig(args["plot"])

# serialize the model to disk
# print("[INFO] saving COVID-19 detector model...")
# model.save(args["model"], save_format="h5")

##Versão modificada com 20 EPOCHS e mais 3 técnicas extras de Data Augmentation

##Análise e Conclusão:
O modelo com 15 épocas e augmentação básica parece ter tido desempenho perfeito, mas é possível que ele tenha sofrido overfitting, aprendendo demais os dados de treino e acertando por sorte em um conjunto pequeno de teste.

Já o modelo com 20 epochs e augmentação mais complexa teve desempenho significativamente inferior. A acurácia caiu para 50%, e tanto a sensibilidade quanto a especificidade foram medianas. Isso pode ter ocorrido por:

Excesso de transformações visuais, dificultando o aprendizado

Aumento da complexidade sem aumento correspondente no tamanho ou diversidade do dataset

Mais epochs sem regularização suficiente, causando instabilidade

Vamos a procurar o caminho de nossa base

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

#Vamos colocar os parametros de nosso modelo (20 epochs)

In [ ]:
INIT_LR = 1e-3
EPOCHS = 20 # Alterado para 20 epochs
BS = 8

# Pega a lista de imagens no nosso diretório de conjunto de dados e, em seguida, inicializa
# a lista de dados e as respectivas classes das imagens

In [ ]:

dataset_path = 'dataset'
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset_path))
data = []
labels = []

In [ ]:
# recorrer las rutas de la imagen
for imagePath in imagePaths:
    # extrae la etiqueta de clase del nombre del archivo
    label = imagePath.split(os.path.sep)[-2]

   # cargue la imagen, intercambie canales de color y cambie su tamaño para que sea una
   # 224x224 píxeles sin tener en cuenta la relación de aspecto
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # actualizar las listas de datos y etiquetas, respectivamente
    data.append(image)
    labels.append(label)
print("labels: ", np.unique(labels))

In [ ]:
# convierta los datos y las etiquetas en matrices NumPy mientras escala el píxel
# intensidades en el rango [0, 255]
data = np.array(data) / 255.0
labels = np.array(labels)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(data[labels=='covid'][1])
plt.show()

In [ ]:
# realizar una codificación one-hot en las etiquetas
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
# particione los datos en divisiones de entrenamiento y prueba usando el 80% de
# los datos para entrenamiento y el 20% restante para pruebas
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.20, stratify=labels, random_state=42)

## Data Augmentation com mais 3 etapas

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),  # adicionei: Aumento de brilho
    channel_shift_range=10.0,  # adicionei: Mudança de cor
    fill_mode='nearest')  # adicionei: Preenchimento de bordas

In [ ]:
# cargue la red VGG16, asegurándose de que queden los conjuntos de capas FC principales
# off
baseModel = VGG16(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
#headModel = MaxPooling2D(pool_size=(2, 2))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)  # Replace 'decay' with 'learning_rate'
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

In [ ]:
baseModel = VGG16(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))

f1 = baseModel.layers[1].output
f2 = baseModel.layers[2].output
f3 = baseModel.layers[4].output
f4 = baseModel.layers[5].output
feature_maps = Model(inputs=baseModel.input, outputs=[f1, f2, f3, f4])

In [ ]:
y = np.argmax(trainY, axis=-1)

In [ ]:
feat1, feat2, feat3, feat4 = feature_maps.predict(trainX[y==1][0:1])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].imshow(feat1[0, :, :, 0:3])
axs[0, 1].imshow(feat2[0, :, :, 6:9])
axs[1, 0].imshow(feat3[0, :, :, 0:3])
axs[1, 1].imshow(feat4[0, :, :, 3:6])
# plt.subplots
# plt.imshow(feat1[0, :, :, 0:3])
plt.show()

##passo os dados de entrada

In [ ]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
    trainAug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
                            target_names=lb.classes_))

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testY.argmax(axis=1), predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()
# plt.savefig(args["plot"])

# serialize the model to disk
# print("[INFO] saving COVID-19 detector model...")
# model.save(args["model"], save_format="h5")

##Teste com VGG19 e comparação com VGG16

In [ ]:
from tensorflow.keras.applications import VGG19

# Carregar a VGG19 com pesos do ImageNet
baseModel_vgg19 = VGG19(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))

# Cabeça da rede
headModel = baseModel_vgg19.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# Modelo completo
model_vgg19 = Model(inputs=baseModel_vgg19.input, outputs=headModel)

# Congelar camadas convolucionais
for layer in baseModel_vgg19.layers:
    layer.trainable = False

# Compilar
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model_vgg19.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Treinar
H19 = model_vgg19.fit(
    trainAug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS
)


In [ ]:
# Avaliação VGG19
predIdxs19 = model_vgg19.predict(testX, batch_size=BS)
predIdxs19 = np.argmax(predIdxs19, axis=1)

print("[INFO] Classification report VGG19:")
print(classification_report(testY.argmax(axis=1), predIdxs19, target_names=lb.classes_))

cm19 = confusion_matrix(testY.argmax(axis=1), predIdxs19)
acc19 = (cm19[0, 0] + cm19[1, 1]) / np.sum(cm19)
sens19 = cm19[0, 0] / (cm19[0, 0] + cm19[0, 1])
spec19 = cm19[1, 1] / (cm19[1, 0] + cm19[1, 1])

print("Matriz de Confusão VGG19:")
print(cm19)
print("Acurácia: {:.4f}".format(acc19))
print("Sensibilidade: {:.4f}".format(sens19))
print("Especificidade: {:.4f}".format(spec19))


In [ ]:
#Comparacao entre VGG16 e VGG19

# make predictions on the testing set for VGG16
print("[INFO] evaluating VGG16 network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report for VGG16
print("[INFO] Classification report VGG16:")
print(classification_report(testY.argmax(axis=1), predIdxs,
                            target_names=lb.classes_))

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity for VGG16
cm = confusion_matrix(testY.argmax(axis=1), predIdxs)
total = sum(sum(cm))
acc16 = (cm[0, 0] + cm[1, 1]) / total
sens16 = cm[0, 0] / (cm[0, 0] + cm[0, 1])
spec16 = cm[1, 1] / (cm[1, 0] + cm[0, 1])

print("Matriz de Confusão VGG16:")
print(cm)
print("Acurácia VGG16: {:.4f}".format(acc16))
print("Sensibilidade VGG16: {:.4f}".format(sens16))
print("Especificidade VGG16: {:.4f}".format(spec16))

# Avaliação VGG19 (assuming model_vgg19, testX, testY, lb are defined in previous cells)
print("[INFO] evaluating VGG19 network...")
predIdxs19 = model_vgg19.predict(testX, batch_size=BS)
predIdxs19 = np.argmax(predIdxs19, axis=1)

print("[INFO] Classification report VGG19:")
print(classification_report(testY.argmax(axis=1), predIdxs19, target_names=lb.classes_))

cm19 = confusion_matrix(testY.argmax(axis=1), predIdxs19)
acc19 = (cm19[0, 0] + cm19[1, 1]) / np.sum(cm19)
sens19 = cm19[0, 0] / (cm19[0, 0] + cm19[0, 1])
spec19 = cm19[1, 1] / (cm19[1, 0] + cm19[1, 1])

print("Matriz de Confusão VGG19:")
print(cm19)
print("Acurácia VGG19: {:.4f}".format(acc19))
print("Sensibilidade VGG19: {:.4f}".format(sens19))
print("Especificidade VGG19: {:.4f}".format(spec19))


# Comparação final
print("\n\n🔍 Comparação entre VGG16 e VGG19")
print("Modelo\t\tAcurácia\tSensibilidade\tEspecificidade")
print(f"VGG16\t\t{acc16:.4f}\t\t{sens16:.4f}\t\t{spec16:.4f}")
print(f"VGG19\t\t{acc19:.4f}\t\t{sens19:.4f}\t\t{spec19:.4f}")

# The plotting code for VGG16 training history is removed from here
# as it was part of the previous evaluation cell for VGG16 and not
# needed for the final comparison table. If you want to plot the
# training history for both, you would do it in separate cells after
# training each model.

#Comparação entre os modelos VGG16 e VGG19

##Acurácia geral:

VGG16 atingiu uma acurácia de 50%

VGG19 obteve 70%, demonstrando melhor desempenho geral na classificação

##Sensibilidade (recall da classe COVID):

Ambos os modelos apresentaram sensibilidade igual (0.4000) para a classe COVID, ou seja, acertaram apenas 2 de 5 imagens positivas

##Especificidade (recall da classe normal):

O VGG16 teve uma especificidade de 60%

O VGG19 alcançou 100%, identificando corretamente todos os casos normais

##Conclusão:

Apesar da sensibilidade baixa em ambos os modelos, o VGG19 se destacou por apresentar acurácia e especificidade superiores.

Isso indica que o VGG19 teve melhor desempenho na identificação de imagens normais e foi mais confiável no contexto geral da classificação, mesmo com um conjunto pequeno de teste.